In [35]:
import numpy as np
import pandas as pd

############################################################################################
path='C://Users//bishal.ghimire//Desktop//data//loadValues.csv'


In [2]:
class DataImport():
    
    def __init__(self,path):
        self.path=path
        self.get_data()
        #self.remove_rstrip()
        self.date_time_format('Time')
       
        pass
    def get_data(self):#reads csv file 
        self.data= pd.read_csv(self.path)
        
    def date_time_format(self,index): #changes a column of a pandas dataframe to date-time format, takes column index as input
        self.data[index]=pd.to_datetime(self.data[index])


In [3]:
class preprocessing():
    
    def __init__(self,pandas_df):
        self.data=pandas_df
        self.interpol()
        self.take_abs()
        #self.scaling()
        self.give_parameter()
        pass
    
    def interpol(self):
        self.data=self.data.interpolate(method='linear')
    def take_abs(self):
        self.data['Consumption']=self.data['Consumption'].abs()
        self.data['Consumption']=self.data['Consumption'].round().astype(int)
        pass
    def scaling(self):
        self.data['Consumption']=self.data['Consumption']/max(self.data['Consumption'])
    def give_parameter(self):
        print(self.data['Consumption'].describe())

    

In [4]:
temp0=DataImport(path)
#print(temp0.data)
temp1=preprocessing(temp0.data)



count    105408.000000
mean        298.048459
std         497.157292
min           0.000000
25%         128.000000
50%         191.000000
75%         275.000000
max       10694.000000
Name: Consumption, dtype: float64


In [5]:
df=temp1.data
df.head()

,Time,Consumption,Unnamed: 2
0,2018-05-01 00:00:00+02:00,117,NaN
1,2018-05-01 00:05:00+02:00,222,NaN
2,2018-05-01 00:10:00+02:00,214,NaN
3,2018-05-01 00:15:00+02:00,620,NaN
4,2018-05-01 00:20:00+02:00,125,NaN


In [6]:
class CountAndReturnCount:
    def __init__(self,df,unique):
        self.df=df
        self.unique=unique
        self.countUniqueinDF()
        self.return_dataset()
    def getUnique(self):
        self.unique=self.df["Consumption"].unique().tolist()
        #print (self.unique)
    def countUniqueinDF(self):
        self.counted=self.df["Consumption"].value_counts()
        self.ferquency=[]
        for i in self.unique:
            try:
                self.ferquency.append(self.counted[i])
            except KeyError:
                self.ferquency.append(0)
    def return_dataset(self):
        
        self.Probablity=pd.DataFrame()
        self.Probablity["Unique"]=self.unique
        self.Probablity["frequency"]=self.ferquency
        self.Probablity["probab"]=self.Probablity["frequency"]/self.Probablity["frequency"].sum()
        self.Probablity= self.Probablity.sort_values(by=['Unique'],ascending=True)
        
        
    
        
    
          
            
        pass
        
    

In [7]:
temp2=CountAndReturnCount(df,df["Consumption"].unique().tolist())
#temp2.Probablity





In [8]:
#temp2.Probablity.plot.scatter('Unique','probab')

In [9]:
train=pd.DataFrame()
train["Time"]=df["Time"][:len(df["Time"])-24*60*5]
train["Consumption"]=df["Consumption"][:len(df["Consumption"])-24*60*5]
validate=pd.DataFrame()
validate["Time"]=df["Time"][len(df["Time"])-48*60*5:len(df["Time"])-24*60*5]
validate["Consumption"]=df["Consumption"][len(df["Consumption"])-48*60*5:len(df["Consumption"])-24*60*5]
test=pd.DataFrame()
test["Time"]=df["Time"][len(df["Time"])-24*60*5:]
test["Consumption"]=df["Consumption"][len(df["Time"])-24*60*5:]


In [10]:
#train
    
    

In [11]:
#test

In [12]:
validate
conditional=[]
train["Time"]=pd.to_datetime(train["Time"],utc=True)
grouped=df.groupby([train['Time'].dt.hour])



In [13]:
a=[]
Conditional=[]

for i in grouped:
    a.append(pd.DataFrame(i[1]))
for j in a:

    temp= CountAndReturnCount(j,df["Consumption"].unique().tolist())
    conditional.append(temp.Probablity)

    
    

In [14]:
class predict():
    def __init__(self,conditional,probablity):
        self.conditional=conditional
        self.df_weight=pd.DataFrame()
        self.probab=probablity       
        self.get_proballity()
    def get_proballity(self):
        self.df_weight["Unique"]=self.probab["Unique"]
        self.df_weight["weight"]=self.conditional['probab']/self.probab['probab']
    def pred(self,df_of_load_with_probablity):
        self.df_weight["predicted"]=df_of_load_with_probablity["probab"]* self.df_weight["weight"]
        higest_probab=self.df_weight['predicted'].max()
        #print( higest_probab)
        consumption_with_higest_probab=self.df_weight.loc[self.df_weight['predicted']==higest_probab,'Unique'].values[0]
        #print( consumption_with_higest_probab)
        
        return consumption_with_higest_probab
        
        
        
        
        

In [15]:
df["Time"]=pd.to_datetime(df["Time"],utc=True)
temp_data1=df.groupby(pd.Grouper(key='Time', freq='24h'))
temp_data2=[]
temp_data3=[]
for i in temp_data1:
    temp_data2.append(i[1])

    


    
    
    

    


In [16]:
for j in temp_data2:
    #j["Time"]=pd.to_datetime(j["Time"],utc=True)
    b=df.groupby(j['Time'].dt.hour)
    temp_data3.append(b)

In [17]:
data_for_prediction=[]
for i in temp_data3:
    temp_data4=[]
    for j in i:
        temp_data4.append(pd.DataFrame(j[1]))
    data_for_prediction.append(temp_data4)
    

In [18]:
#calculating probablity of each elememt with reference to unique element and creating a dat frame

In [19]:
print(len(conditional))

temp6=CountAndReturnCount(data_for_prediction[1][3],df["Consumption"].unique().tolist())


24


In [20]:
temp6.Probablity.max()

Unique       10694.000000
frequency        1.000000
probab           0.083333
dtype: float64

In [21]:
result_final=[]
print(len(data_for_prediction),len( data_for_prediction[1][0]))
for j in range(len(data_for_prediction)):
    result=[]
    for i in range (len( data_for_prediction[j])):
        #print(j,i)
        try:
            temp6=CountAndReturnCount(data_for_prediction[j][i],df["Consumption"].unique().tolist())
            temp7=predict(conditional[i],temp6.Probablity)
            #print(temp7.df_weight)
            result.append(temp7.pred(temp6.Probablity))
    
        except IndexError :
            print('error occur')
    
            pass
    result_final.append(result)
#len(result)

367 12


In [22]:
temp6.Probablity.max()

Unique       10694.000000
frequency        1.000000
probab           0.083333
dtype: float64

In [26]:
result_all=pd.DataFrame()

result_all

""


In [27]:
result_final

[[108, 120],
 [116,
  108,
  114,
  94,
  131,
  131,
  197,
  132,
  0,
  184,
  0,
  158,
  155,
  180,
  158,
  157,
  157,
  140,
  0,
  113,
  110,
  190,
  140,
  118],
 [0,
  101,
  113,
  110,
  0,
  0,
  125,
  0,
  187,
  206,
  277,
  169,
  169,
  191,
  175,
  135,
  137,
  194,
  221,
  119,
  220,
  110,
  98,
  104],
 [110,
  101,
  116,
  250,
  89,
  150,
  122,
  133,
  107,
  162,
  136,
  160,
  172,
  150,
  148,
  170,
  150,
  56,
  118,
  181,
  267,
  119,
  106,
  110],
 [111,
  77,
  100,
  220,
  242,
  148,
  140,
  126,
  118,
  167,
  315,
  240,
  178,
  225,
  190,
  157,
  137,
  74,
  180,
  150,
  160,
  180,
  110,
  108],
 [100,
  113,
  121,
  207,
  136,
  122,
  120,
  126,
  186,
  187,
  160,
  160,
  172,
  170,
  184,
  144,
  137,
  219,
  194,
  140,
  99,
  111,
  130,
  115],
 [108,
  99,
  75,
  100,
  115,
  237,
  201,
  135,
  168,
  162,
  178,
  282,
  292,
  301,
  165,
  229,
  169,
  35,
  173,
  139,
  110,
  104,
  97,
  108]

In [32]:
orginal_avrage=[]
for i in grouped:
    orginal_avrage.append(i[1]['Consumption'].mean())

In [34]:
orginal_avrage

[172.6458944281525,
 181.0823558162268,
 247.38563049853371,
 282.61534701857283,
 236.39271749755622,
 251.62487781036168,
 238.14418377321604,
 238.43157380254155,
 232.83284457478007,
 276.4015151515151,
 324.39760508308893,
 372.80474095796677,
 416.5923753665689,
 340.4379276637341,
 347.25171065493646,
 374.61094819159337,
 427.1204789833822,
 445.3453079178886,
 392.1297653958944,
 354.7705278592375,
 290.219696969697,
 257.2465786901271,
 231.1669110459433,
 200.4037145650049]